### NumPy og Pandas er brukt til å beregne gjennomsnitt, median og standardavvik.


In [15]:
import numpy as np
import pandas as pd
import requests

# Hent data fra NASA Sea Level API
response = requests.get(
    "https://sealevel-nexus.jpl.nasa.gov/timeSeriesSpark",
    params={
        "ds": "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly",
        "b": "-180.0,-90.0,180.0,90.0",
        "startTime": "1992-01-01T00:00:00Z",
        "endTime": "2025-01-01T00:00:00Z",
        "output": "JSON"
    }
)

# Sjekk at responsen er god
if response.ok and "data" in response.json():
    raw_data = [item for sublist in response.json()["data"] for item in sublist]
    df = pd.DataFrame(raw_data)
else:
    raise ValueError("Feil ved henting av data.")

# Filtrer gyldige havnivåverdier
valid_mean = df['mean'].dropna()
valid_mean = valid_mean[valid_mean.between(-0.1, 0.1)]
invalid_count = len(df) - len(valid_mean)

# Beregn statistiske mål med NumPy
gjennomsnitt = np.mean(valid_mean)
median = np.median(valid_mean)
std_avvik = np.std(valid_mean)

# Print resultatene
print(f"Antall usaklige verdier: {invalid_count}")
print(f"\nGjennomsnitt: {gjennomsnitt}")
print(f"Median: {median}")
print(f"Standardavvik: {std_avvik}")


Antall usaklige verdier: 0

Gjennomsnitt: 0.029373403279491547
Median: 0.024730028380728995
Standardavvik: 0.02889329956410606


### Undersøker korrelasjon mellom havnivåstigning og tid ved bruk av Pearson-korrelasjon. 
En korrelasjon nær 1 indikerer en sterk positiv sammenheng (havnivået øker over tid). En korrelasjon nær -1 indikerer en sterk negativ sammenheng (havnivået synker). En korrelasjon nær 0 betyr at det ikke er noen tydelig sammenheng.

In [2]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from scipy.stats import pearsonr

# (Samme hentekode som før...)
response = requests.get(
    "https://sealevel-nexus.jpl.nasa.gov/timeSeriesSpark",
    params={
        "ds": "NASA_SSH_REF_SIMPLE_GRID_V1_Monthly",
        "b": "-180.0,-90.0,180.0,90.0",
        "startTime": "1992-01-01T00:00:00Z",
        "endTime": "2025-01-01T00:00:00Z",
        "output": "JSON"
    }
)

if response.ok and "data" in response.json():
    raw_data = [item for sublist in response.json()["data"] for item in sublist]
    df = pd.DataFrame(raw_data)
else:
    raise ValueError("Feil ved henting av data.")

# Filtrer og forbered data
df = df.dropna(subset=["mean", "iso_time"])
df = df[df["mean"].between(-0.1, 0.1)]

# Gjør tid om til tall (for eksempel antall dager siden start)
df["iso_time"] = pd.to_datetime(df["iso_time"])
df["tid_i_dager"] = (df["iso_time"] - df["iso_time"].min()).dt.days

# Beregn korrelasjon mellom tid og havnivå
korrelasjon, _ = pearsonr(df["tid_i_dager"], df["mean"])
print(f"Korrelasjon mellom tid og havnivå: {korrelasjon}")


Korrelasjon mellom tid og havnivå: 0.9758431979304274
